# Project Setup

## Installing Required Libraries

Installing the libraries required by this project

In [1]:
!pip install tensorflow
!pip install tensorflow-hub
!pip install scikit-learn
!pip install keras-tuner
!pip install seaborn 
!pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Data Preprocessing

It was noted that some of the folders, in particular the art style ukiyo e had different names for the folders across the Latend Diffusion, Stable DIffusion and Human Samples. Considerations we made and the folders were renamed "ukiyo-e" across the test and train samples to help with the functions created to auto train and test

## Dataset Preparation

Creating a reference to the base location of all the files

In [2]:
import os #used in this instance to get the relative location of the notebook

#set the base directory NB windows using relative paths causes errors
BASE_LOC = os.getcwd();
print(BASE_LOC)#ensure current directory is not relative

/home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset


Creating a function to return a file list from a particular directory

In [3]:
import os
import random
import pandas as pd

def generate_image_df(directory, label, sample_size=None):
    data = {'filepath': [], 'label': []}

    #collect file paths and labels
    for file in os.listdir(directory):
        filepath = os.path.join(directory, file)
        data['filepath'].append(filepath)
        data['label'].append(label)

    #if we give a specified # of samples to use
    if sample_size is not None:
        if sample_size >= len(data['filepath']):
            return pd.DataFrame(data)  #return all if sample_size is greater or equal to the number of files
        else:
            indices = random.sample(range(len(data['filepath'])), sample_size)
            sampled_data = {key: [data[key][i] for i in indices] for key in data}
            return pd.DataFrame(sampled_data)

    return pd.DataFrame(data)

Creating a function to prepare a dataset based on the above function returning a list for each of the particular generation styles of a singe art style

In [4]:
def artstyle_dataset(train_or_test, art_style, sample_size):
    file_location = os.path.join(BASE_LOC, 'train')

    if train_or_test == 'test':
        file_location = os.path.join(BASE_LOC, 'test')

    # half the sample size for each AI style
    ai_hum_samp = sample_size // 2

    # get human samples
    human_location = os.path.join(file_location, art_style)
    print("Location of human samples:", human_location)
    human_data = generate_image_df(human_location, "HU", sample_size)

    ai_sd_location = os.path.join(file_location, 'AI_SD_' + art_style)
    print("Location of ai_sd samples:", ai_sd_location)
    ai_sd_data = generate_image_df(ai_sd_location, 'AI', ai_hum_samp)

    ai_ld_location = os.path.join(file_location, 'AI_SD_' + art_style)
    print("Location of ai_ld samples:", ai_ld_location)
    ai_ld_data = generate_image_df(ai_ld_location, 'AI', ai_hum_samp)

    # Combine human and AI data
    dataset = pd.concat([human_data, ai_sd_data, ai_ld_data], ignore_index=True)
    
    return dataset

Testing the genertae_image_df function

In [5]:
ukiyoe_tr_1_10 = artstyle_dataset('train', 'ukiyo-e', 500)
ukiyoe_1_10_file_counts = ukiyoe_tr_1_10['label'].value_counts()
#print value count
print(f"\nLABEL COUNTS ON ukiyoe_tr_1_10 TRAINING SET")
print(ukiyoe_1_10_file_counts)
print('\n')

Location of human samples: /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/train/ukiyo-e
Location of ai_sd samples: /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/train/AI_SD_ukiyo-e
Location of ai_ld samples: /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/train/AI_SD_ukiyo-e

LABEL COUNTS ON ukiyoe_tr_1_10 TRAINING SET
label
HU    500
AI    500
Name: count, dtype: int64




Clearing the variables to clear memeory

In [6]:
del ukiyoe_tr_1_10
del ukiyoe_1_10_file_counts

## CNN Setup

In [7]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping

def CNN_artystyle(filters=64, kernel_size=3, input_shape=(240, 240, 3), dense_units=64, output_units=2, activation='relu'):
    model = Sequential()
    model.add(Conv2D(filters=filters, kernel_size=kernel_size, activation=activation, input_shape=input_shape))
    model.add(MaxPooling2D())
    model.add(Conv2D(filters=filters, kernel_size=kernel_size, activation=activation))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(units=dense_units, activation=activation))
    model.add(Dropout(0.5))  # Add dropout layer to reduce overfitting
    model.add(Dense(units=output_units, activation='sigmoid'))
    
    # Configure early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    return model, early_stopping

2024-04-19 01:08:06.509160: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-19 01:08:08.238890: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-19 01:08:11.468445: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-19 01:08:19.378624: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Creating the training function

In [8]:
import time

def CNN_img_TRAIN_MODEL(model, training_dataset, validation_dataset, model_checkpoint, early_stopping, epochs=13):
    #init training time
    total_training_time = 0

    start_time = time.time()

    #fit the model for the specified number of epochs
    history = model.fit(training_dataset, epochs=epochs, validation_data=validation_dataset, callbacks=[model_checkpoint, early_stopping])

    #calculate training time for the current batch of epochs
    training_time = time.time() - start_time
    total_training_time += training_time

    #print total training time
    print("Total training time so far: {:.2f} seconds".format(total_training_time))

    return history

# Auto Train Function

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

def CNN_train_artstyle(artstyle, sample_size=None, validation_split=0.2):
    #creating the training set
    init_dataset = artstyle_dataset('train', artstyle, sample_size)
    init_dataset_counts = init_dataset['label'].value_counts()
    #print value count
    print(f"\nLABEL COUNTS ON {artstyle} TRAINING SET")
    print(init_dataset_counts)
    print('\n')

    #splitting the init dataset into train and validation
    from sklearn.model_selection import train_test_split

    train, vali = train_test_split(init_dataset, test_size=validation_split, random_state=42)

    #preprocess training set
    train_dataset_gen = ImageDataGenerator(
        rescale=1./255,   # to normalize pixel value
        #rotation_range=7, # it will apply rotations to the image
        #horizontal_flip=True, # it will flip image horizontally
        #zoom_range=0.2  # it will increase and decrease zoom by 0.2x
    )

    train_dataset = train_dataset_gen.flow_from_dataframe(
        dataframe=train,
        x_col='filepath',  # Column containing file paths
        y_col='label',     # Column containing labels
        target_size=(240, 240),
        batch_size=64,
        class_mode='categorical',  
        shuffle=True
    )

    print (train_dataset.class_indices)

    #preprocess training set
    vali_dataset_gen = ImageDataGenerator(
        rescale=1./255,   # to normalize pixel value
        rotation_range=7, # it will apply rotations to the image
        horizontal_flip=True, # it will flip image horizontally
        zoom_range=0.2  # it will increase and decrease zoom by 0.2x
    )

    vali_dataset = vali_dataset_gen.flow_from_dataframe(
        dataframe=vali,
        x_col='filepath',  # Column containing file paths
        y_col='label',     # Column containing labels
        target_size=(240, 240),
        batch_size=64,
        class_mode='categorical',  
        shuffle=True
    )

    print (vali_dataset.class_indices)

    #create the model
    CNN_model, CNN_model_ES = CNN_artystyle()

    #create checkpoint
    os.makedirs('models_checkpoints', exist_ok=True)
    model_file_name = f'CNN_{artstyle}'
    
    #create location for check point
    current_dir = os.getcwd()
    filename = (f'models_checkpoints/{model_file_name}.keras')
    checkpoint_filepath = os.path.join(current_dir, filename)

    #create a checkpoint
    CNN_model_CheckPoint = ModelCheckpoint(checkpoint_filepath, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

    #compile the model
    # Compile the model
    learning_rate = 1e-4
    optimizer = Adam()
    CNN_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    #train the model
    model_history = CNN_img_TRAIN_MODEL(CNN_model, train_dataset, vali_dataset, CNN_model_CheckPoint, CNN_model_ES)

    #save model after training
    os.makedirs('models_trained', exist_ok=True)
    CNN_model.save(f'models_trained/{model_file_name}.keras')

    return model_history


Save history as a json file

In [10]:
import json
import os
import numpy as np

def save_training_history(history, filename):
    # Convert non-serializable parts to serializable format
    serializable_history = {}
    for key, value in history.items():
        if isinstance(value, np.ndarray):
            serializable_history[key] = value.tolist()
        else:
            serializable_history[key] = value

    directory_path = 'models_history'
    curr_dir = os.getcwd()
    full_dir_path = os.path.join(curr_dir, directory_path)

    if not os.path.exists(full_dir_path):
        os.makedirs(full_dir_path)

    file_path = os.path.join(full_dir_path, filename)

    with open(file_path, 'w') as file:
        json.dump(serializable_history, file)
    
    print('Training history saved to', file_path)

In [9]:
#ukiyo_history = CNN_train_artstyle('ukiyo-e', sample_size=4500)

Location of human samples: /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/train/ukiyo-e
Location of ai_sd samples: /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/train/AI_SD_ukiyo-e
Location of ai_ld samples: /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/train/AI_SD_ukiyo-e

LABEL COUNTS ON ukiyo-e TRAINING SET
label
HU    4500
AI    4500
Name: count, dtype: int64


Found 7200 validated image filenames belonging to 2 classes.
{'AI': 0, 'HU': 1}
Found 1799 validated image filenames belonging to 2 classes.
{'AI': 0, 'HU': 1}


/home/g0dbot/.local/lib/python3.10/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 1 invalid image filename(s) in x_col="filepath". These filename(s) will be ignored.
  warnings.warn(
/home/g0dbot/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/13


/home/g0dbot/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-04-18 20:18:34.667412: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 928055296 exceeds 10% of free system memory.
2024-04-18 20:18:37.882412: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 464027648 exceeds 10% of free system memory.
2024-04-18 20:18:37.882544: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 928055296 exceeds 10% of free system memory.


  1/113 ━━━━━━━━━━━━━━━━━━━━ 20:37 11s/step - accuracy: 0.4375 - loss: 0.7159

2024-04-18 20:18:40.234773: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 928055296 exceeds 10% of free system memory.
2024-04-18 20:18:43.429091: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 464027648 exceeds 10% of free system memory.


113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.8189 - loss: 0.8263
Epoch 1: val_accuracy improved from -inf to 0.66148, saving model to /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/models_checkpoints/CNN_ukiyo-e.keras
113/113 ━━━━━━━━━━━━━━━━━━━━ 700s 6s/step - accuracy: 0.8199 - loss: 0.8221 - val_accuracy: 0.6615 - val_loss: 0.7512
Epoch 2/13
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9849 - loss: 0.1203
Epoch 2: val_accuracy improved from 0.66148 to 0.69594, saving model to /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/models_checkpoints/CNN_ukiyo-e.keras
113/113 ━━━━━━━━━━━━━━━━━━━━ 699s 6s/step - accuracy: 0.9848 - loss: 0.1204 - val_accuracy: 0.6959 - val_loss: 0.7315
Epoch 3/13
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9947 - loss: 0.0848
Epoch 3: val_accuracy improved from 0.69594 to 0.84825, saving model to /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/models_checkpoints/CNN_ukiyo-e.keras
1

In [22]:
#save_training_history(ukiyo_history.history, 'ukiyoe_history.json')

Training history saved to /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/models_history/ukiyoe_history.json


In [10]:
#realism_history = CNN_train_artstyle('realism', sample_size=4500)

Location of human samples: /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/train/realism
Location of ai_sd samples: /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/train/AI_SD_realism
Location of ai_ld samples: /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/train/AI_SD_realism

LABEL COUNTS ON realism TRAINING SET
label
HU    4500
AI    4500
Name: count, dtype: int64


Found 7200 validated image filenames belonging to 2 classes.
{'AI': 0, 'HU': 1}
Found 1800 validated image filenames belonging to 2 classes.
{'AI': 0, 'HU': 1}


/home/g0dbot/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/13


/home/g0dbot/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-04-18 21:55:56.529170: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:27: Filling up shuffle buffer (this may take a while): 5 of 8
2024-04-18 21:56:09.119440: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.7046 - loss: 0.7914
Epoch 1: val_accuracy improved from -inf to 0.81500, saving model to /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/models_checkpoints/CNN_realism.keras
113/113 ━━━━━━━━━━━━━━━━━━━━ 727s 6s/step - accuracy: 0.7054 - loss: 0.7891 - val_accuracy: 0.8150 - val_loss: 0.3703
Epoch 2/13
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.8648 - loss: 0.3476
Epoch 2: val_accuracy improved from 0.81500 to 0.88222, saving model to /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/models_checkpoints/CNN_realism.keras
113/113 ━━━━━━━━━━━━━━━━━━━━ 702s 6s/step - accuracy: 0.8649 - loss: 0.3473 - val_accuracy: 0.8822 - val_loss: 0.2705
Epoch 3/13
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9145 - loss: 0.2324
Epoch 3: val_accuracy improved from 0.88222 to 0.91444, saving model to /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/models_checkpoints/CNN_realism.keras
1

In [23]:
#save_training_history(realism_history.history, 'realism_history.json')

Training history saved to /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/models_history/realism_history.json


### Baroque

In [11]:
baroque_history = CNN_train_artstyle('baroque', sample_size=4500)
save_training_history(baroque_history.history, 'baroque_history.json')

Location of human samples: /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/train/baroque
Location of ai_sd samples: /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/train/AI_SD_baroque
Location of ai_ld samples: /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/train/AI_SD_baroque

LABEL COUNTS ON baroque TRAINING SET
label
HU    4500
AI    4500
Name: count, dtype: int64


Found 7200 validated image filenames belonging to 2 classes.
{'AI': 0, 'HU': 1}
Found 1800 validated image filenames belonging to 2 classes.
{'AI': 0, 'HU': 1}


/home/g0dbot/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/13


/home/g0dbot/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-04-19 01:08:59.085354: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 928055296 exceeds 10% of free system memory.
2024-04-19 01:09:03.587112: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 464027648 exceeds 10% of free system memory.
2024-04-19 01:09:03.587208: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 928055296 exceeds 10% of free system memory.


  1/113 ━━━━━━━━━━━━━━━━━━━━ 37:26 20s/step - accuracy: 0.4375 - loss: 0.6998

2024-04-19 01:09:08.840357: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 928055296 exceeds 10% of free system memory.
2024-04-19 01:09:12.609238: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 464027648 exceeds 10% of free system memory.


113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.7660 - loss: 0.6260
Epoch 1: val_accuracy improved from -inf to 0.79167, saving model to /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/models_checkpoints/CNN_baroque.keras
113/113 ━━━━━━━━━━━━━━━━━━━━ 735s 6s/step - accuracy: 0.7671 - loss: 0.6229 - val_accuracy: 0.7917 - val_loss: 0.4503
Epoch 2/13
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9724 - loss: 0.0899
Epoch 2: val_accuracy did not improve from 0.79167
113/113 ━━━━━━━━━━━━━━━━━━━━ 703s 6s/step - accuracy: 0.9724 - loss: 0.0898 - val_accuracy: 0.5183 - val_loss: 1.0017
Epoch 3/13
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9883 - loss: 0.0481
Epoch 3: val_accuracy did not improve from 0.79167
113/113 ━━━━━━━━━━━━━━━━━━━━ 688s 6s/step - accuracy: 0.9883 - loss: 0.0480 - val_accuracy: 0.5372 - val_loss: 2.0935
Epoch 4/13
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9963 - loss: 0.0160
Epoch 4: val_accuracy did not improve from 0

### Expressionism

In [12]:
expressionism_history = CNN_train_artstyle('expressionism', sample_size=4500)
save_training_history(expressionism_history.history, 'expressionism_history.json')

Location of human samples: /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/train/expressionism
Location of ai_sd samples: /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/train/AI_SD_expressionism
Location of ai_ld samples: /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/train/AI_SD_expressionism

LABEL COUNTS ON expressionism TRAINING SET
label
HU    4500
AI    4500
Name: count, dtype: int64


Found 7200 validated image filenames belonging to 2 classes.
{'AI': 0, 'HU': 1}
Found 1800 validated image filenames belonging to 2 classes.
{'AI': 0, 'HU': 1}


/home/g0dbot/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/13


/home/g0dbot/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-04-19 02:18:59.032458: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:23: Filling up shuffle buffer (this may take a while): 7 of 8
2024-04-19 02:19:00.183223: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9030 - loss: 0.3833
Epoch 1: val_accuracy improved from -inf to 0.98056, saving model to /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/models_checkpoints/CNN_expressionism.keras
113/113 ━━━━━━━━━━━━━━━━━━━━ 771s 7s/step - accuracy: 0.9036 - loss: 0.3810 - val_accuracy: 0.9806 - val_loss: 0.0831
Epoch 2/13
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9913 - loss: 0.0330
Epoch 2: val_accuracy improved from 0.98056 to 0.99667, saving model to /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/models_checkpoints/CNN_expressionism.keras
113/113 ━━━━━━━━━━━━━━━━━━━━ 747s 7s/step - accuracy: 0.9914 - loss: 0.0329 - val_accuracy: 0.9967 - val_loss: 0.0139
Epoch 3/13
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9987 - loss: 0.0062
Epoch 3: val_accuracy did not improve from 0.99667
113/113 ━━━━━━━━━━━━━━━━━━━━ 772s 6s/step - accuracy: 0.9987 - loss: 0.0062 - val_accuracy: 0.9967 - val_loss: 0

### Impressionism

In [13]:
impressionism_history = CNN_train_artstyle('impressionism', sample_size=4500)
save_training_history(impressionism_history.history, 'impressionism_history.json')

Location of human samples: /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/train/impressionism
Location of ai_sd samples: /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/train/AI_SD_impressionism
Location of ai_ld samples: /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/train/AI_SD_impressionism

LABEL COUNTS ON impressionism TRAINING SET
label
HU    4500
AI    4500
Name: count, dtype: int64


Found 7200 validated image filenames belonging to 2 classes.
{'AI': 0, 'HU': 1}
Found 1800 validated image filenames belonging to 2 classes.
{'AI': 0, 'HU': 1}


/home/g0dbot/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/13


/home/g0dbot/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.8896 - loss: 0.4874
Epoch 1: val_accuracy improved from -inf to 0.99222, saving model to /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/models_checkpoints/CNN_impressionism.keras
113/113 ━━━━━━━━━━━━━━━━━━━━ 705s 6s/step - accuracy: 0.8903 - loss: 0.4846 - val_accuracy: 0.9922 - val_loss: 0.0265
Epoch 2/13
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9923 - loss: 0.0308
Epoch 2: val_accuracy did not improve from 0.99222
113/113 ━━━━━━━━━━━━━━━━━━━━ 677s 6s/step - accuracy: 0.9923 - loss: 0.0308 - val_accuracy: 0.9922 - val_loss: 0.0314
Epoch 3/13
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9941 - loss: 0.0260
Epoch 3: val_accuracy improved from 0.99222 to 0.99444, saving model to /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/models_checkpoints/CNN_impressionism.keras
113/113 ━━━━━━━━━━━━━━━━━━━━ 678s 6s/step - accuracy: 0.9941 - loss: 0.0259 - val_accuracy: 0.9944 - val_loss: 0

### Post Impressionism

In [14]:
post_impressionism_history = CNN_train_artstyle('post_impressionism', sample_size=4500)
save_training_history(post_impressionism_history.history, 'post_impressionism_history.json')

Location of human samples: /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/train/post_impressionism
Location of ai_sd samples: /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/train/AI_SD_post_impressionism
Location of ai_ld samples: /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/train/AI_SD_post_impressionism

LABEL COUNTS ON post_impressionism TRAINING SET
label
HU    4500
AI    4500
Name: count, dtype: int64


Found 7200 validated image filenames belonging to 2 classes.
{'AI': 0, 'HU': 1}
Found 1800 validated image filenames belonging to 2 classes.
{'AI': 0, 'HU': 1}


/home/g0dbot/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/13


/home/g0dbot/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-04-19 06:12:54.740037: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:81: Filling up shuffle buffer (this may take a while): 7 of 8
2024-04-19 06:12:56.534432: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.8995 - loss: 0.5325
Epoch 1: val_accuracy improved from -inf to 0.94500, saving model to /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/models_checkpoints/CNN_post_impressionism.keras
113/113 ━━━━━━━━━━━━━━━━━━━━ 670s 6s/step - accuracy: 0.9001 - loss: 0.5293 - val_accuracy: 0.9450 - val_loss: 0.1368
Epoch 2/13
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9954 - loss: 0.0153
Epoch 2: val_accuracy improved from 0.94500 to 0.95056, saving model to /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/models_checkpoints/CNN_post_impressionism.keras
113/113 ━━━━━━━━━━━━━━━━━━━━ 750s 7s/step - accuracy: 0.9954 - loss: 0.0153 - val_accuracy: 0.9506 - val_loss: 0.1781
Epoch 3/13
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9791 - loss: 0.0766
Epoch 3: val_accuracy improved from 0.95056 to 0.99333, saving model to /home/g0dbot/Desktop/COMP3610/proj/archive/Real_AI_SD_LD_Dataset/models_checkpoin

### Renaissance

In [15]:
#renaissance_history = CNN_train_artstyle('renaissance', sample_size=4500)
#save_training_history(renaissance_history.history, 'renaissance_history.json')

### Romanticism

In [16]:
#romanticism_history = CNN_train_artstyle('romanticism', sample_size=4500)
#save_training_history(romanticism_history.history, 'romanticism_history.json')

### Surrealism

In [17]:
#surrealism_history = CNN_train_artstyle('surrealism', sample_size=4500)
#save_training_history(surrealism_history.history, 'surrealism_history.json')

### Art Nouveau

In [18]:
#art_nouveau_history = CNN_train_artstyle('art_nouveau', sample_size=4500)
#save_training_history(art_nouveau_history.history, 'art_nouveau_history.json')